# Ex 5

In [32]:
# Imports
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

In [33]:
#### Load Data
df1 = pd.read_csv('./data.csv')
df1.rename(columns = {
    'v1_miss':'used_good',
    'v2_miss':'saftey',
    'v3_miss':'celebrities',
    'v4_miss':'foreign_good',
    'v5_miss':'rarely_sit'},
         inplace = True)
df1.head()

,used_good,saftey,celebrities,foreign_good,rarely_sit
0,4.0,3.0,1.0,3.0,1.0
1,2.0,4.0,1.0,NaN,3.0
2,3.0,3.0,3.0,3.0,NaN
3,2.0,4.0,1.0,2.0,2.0
4,2.0,4.0,2.0,5.0,5.0


## 1.	Produce means and standard deviations in a table for the five variables in the data set.

In [45]:
mean = pd.DataFrame(df1.mean(), columns=['NAs'])
std = pd.DataFrame(df1.std(), columns = ['NAs'])
print('MEAN')
mean.T

MEAN


,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,3.128788,3.366412,1.976562,2.20155,2.178862


In [46]:
print('STD')
std.T

STD


,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs_std,1.213193,1.144929,1.090148,1.134542,1.293315


## 2.	Replace the missing values for all the variables in the data set using the mean substitution method.  Place that table here…

In [50]:
df2 = df1.fillna(df1.mean())

mean['mean'] = df2.mean()
std['mean'] = df2.std()
mean.T

,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,3.128788,3.366412,1.976562,2.20155,2.178862
mean,3.128788,3.366412,1.976562,2.20155,2.178862


In [51]:
print('STD')
std.T

,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,1.141391,1.073049,1.009849,1.055102,1.174231
mean,1.141391,1.073049,1.009849,1.055102,1.174231


## 3.	Compare the means and standard deviations from the original data set table with the mean substitution table.  What should happen to the means?  What should happen to the standard deviations?

- The means should stay the same and the stds should shrink as you are artificially decreasing the variance in the data. This is in fact what is happening

## 4.	Run one simple non-multiple imputation models (e.g. just a simple regression model) –  for v3_miss.  Be sure to replace the missing values in v3_miss with the predicted values from the regression.  (I shortened this to just one variable – too much work otherwise I think).

In [36]:
# this is code for all variables. 

In [37]:
lr_df = df1.copy()
for col in lr_df.columns:
    # Calculate mean for all columns except the specific column
    mean_values = lr_df.drop(columns=[col]).mean()

    # Fill missing values with mean for all columns except the specific column
    non_cel_filled_means = df1.fillna(mean_values)

    train = non_cel_filled_means[~non_cel_filled_means[col].isna()]
    X_train = train.drop(col, axis = 1)
    y_train = train[col]

    test = non_cel_filled_means[non_cel_filled_means[col].isna()]

    X_test = test.drop(col, axis = 1)

    # Fit the regression model
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # Predict missing 'celebrities' values
    preds = lr.predict(X_test)
    
    # Iterate over the index of the 'test' DataFrame and corresponding predictions
    for i, pred in zip(test.index, preds):
        lr_df.loc[i, col] = pred

In [38]:
# below is code for one variable

In [39]:
# # Calculate mean for all columns except the specific column
# mean_values = df1.drop(columns=['celebrities']).mean()

# # Fill missing values with mean for all columns except the specific column
# non_cel_filled_means = df1.fillna(mean_values)

# train = non_cel_filled_means[~non_cel_filled_means['celebrities'].isna()]
# X_train = train.drop('celebrities', axis = 1)
# y_train = train['celebrities']

# test = non_cel_filled_means[non_cel_filled_means['celebrities'].isna()]

# X_test = test.drop('celebrities', axis = 1)

# # Fit the regression model
# lr.fit(X_train, y_train)

# # Predict missing 'celebrities' values
# preds = lr.predict(X_test)

# # Iterate over the index of the 'test' DataFrame and corresponding predictions
# for i, pred in zip(test.index, preds):
#     df3.loc[i, 'celebrities'] = pred

## 5.	Compare the table of means and standard deviations from the original data set to the one constructed in step 4 above.  What do you see?

In [54]:
mean['lr'] = lr_df.mean()
std['lr'] = lr_df.std()
print('MEAN')
mean.T

MEAN


,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,3.128788,3.366412,1.976562,2.20155,2.178862
mean,3.128788,3.366412,1.976562,2.20155,2.178862
lr,3.128491,3.367260,1.990286,2.19883,2.195361


In [55]:
print('STD')
std.T

STD


,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,1.141391,1.073049,1.009849,1.055102,1.174231
mean,1.141391,1.073049,1.009849,1.055102,1.174231
lr,1.143532,1.073814,1.017997,1.062925,1.192424


## 6.	Run a multiple imputation model on the five variables – you can pick from multiple imputation regression, mcmc or any other flavor.

In [41]:
from fancyimpute import IterativeImputer

# Create an instance of IterativeImputer
chain = IterativeImputer()

# Fit the imputer on your dataset
chain.fit(df1)

# Impute missing values
chain_df = chain.transform(df1)

In [56]:
mean['chain_mean'] = chain_df.mean()
std['chain_std'] = chain_df.std()
print('MEAN')
mean.T

MEAN


,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,3.128788,3.366412,1.976562,2.201550,2.178862
mean,3.128788,3.366412,1.976562,2.201550,2.178862
lr,3.128491,3.367260,1.990286,2.198830,2.195361
chain_mean,2.576619,2.576619,2.576619,2.576619,2.576619


In [57]:
print('STD')
std.T

STD


,used_good,saftey,celebrities,foreign_good,rarely_sit
NAs,1.141391,1.073049,1.009849,1.055102,1.174231
mean,1.141391,1.073049,1.009849,1.055102,1.174231
lr,1.143532,1.073814,1.017997,1.062925,1.192424
chain_std,1.229168,1.229168,1.229168,1.229168,1.229168


## 7.	Compare the table of means and standard deviations from step 6 with the original data set.  What do you expect?  What do you actually see?

## 8.	For an extra 8 points extra credit, produce a missing values matrix like the one shown in class.  You can use SAS or name your poison in terms of platform as long as it produces a similar matrix.